# Imports

In [155]:
from preprocessing import *

# Data Setup

## Load Data

In [156]:
data_dir = 'project_data/project/'
X_train_valid, y_train_valid, X_test, y_test = load_data(data_dir, subjects=[1]) # default subjects=[1,2,3,4,5,6,7,8,9]

## Preprocessing

In [157]:
x_train, y_train, x_valid, y_valid, x_test, y_test = main_prep(X_train_valid,y_train_valid,X_test, y_test,2,2,True)

Shape of x_train: (758, 250, 1, 22)
Shape of x_valid: (190, 250, 1, 22)
Shape of x_test: (200, 250, 1, 22)
Shape of y_train: torch.Size([758, 4])
Shape of y_valid: torch.Size([190, 4])
Shape of y_test: torch.Size([200, 4])


## PyTorch Dataloaders

In [158]:
trainloader, validloader, testloader = dataloader_setup(x_train, y_train, x_valid, y_valid, x_test, y_test, batch_size=32)

# CNN 

In [179]:
import torch.nn as nn 
import torch.nn.functional as F 


class CNN(nn.Module):
    
    def __init__(self):
        super().__init__()
        #conv layers
        # 250: input channels, 10: output channels, 5x5: square convolution kernel
        self.conv1 = nn.Conv2d(22, 25, 1)
        self.conv2 = nn.Conv2d(250, 25, 1)
        self.pool = nn.MaxPool2d(1, stride=3)
        self.conv3 = nn.Conv2d(25, 50, 1)
        ''
        # affine layers
        self.fc1 = nn.Linear(150, 400)
        self.fc2 = nn.Linear(400, 120)
        self.fc3 = nn.Linear(120, 80)
        self.fc4 = nn.Linear(80, 4) # 4 for output classes

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = self.pool(F.relu(self.conv2(torch.transpose(x, 1, 3))))
        x = self.pool(F.relu(self.conv3(x)))
        x = torch.flatten(x, 1) # flatten all dimensions except batch
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        x = self.fc4(x)
        return x


cnn = CNN()

In [180]:
print(cnn)

CNN(
  (conv1): Conv2d(22, 25, kernel_size=(1, 1), stride=(1, 1))
  (conv2): Conv2d(250, 25, kernel_size=(1, 1), stride=(1, 1))
  (pool): MaxPool2d(kernel_size=1, stride=3, padding=0, dilation=1, ceil_mode=False)
  (conv3): Conv2d(25, 50, kernel_size=(1, 1), stride=(1, 1))
  (fc1): Linear(in_features=150, out_features=400, bias=True)
  (fc2): Linear(in_features=400, out_features=120, bias=True)
  (fc3): Linear(in_features=120, out_features=80, bias=True)
  (fc4): Linear(in_features=80, out_features=4, bias=True)
)


## CNN Optimizer and Loss

In [181]:
import torch.optim as optim

criterion = nn.CrossEntropyLoss()

# create your optimizer
optimizer = optim.Adam(cnn.parameters(), lr=0.001)

In [182]:
EPOCHS = 100

for epoch in range(EPOCHS):

    running_loss = 0.0
    loss = None
    for i, data in enumerate(trainloader, 0):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data
        labels = labels.float()

        
        # reshape inputs for time series convolution
        inputs = torch.transpose(inputs, 1, 3)
  
        # forward pass
        outputs = cnn(inputs)
        loss = criterion(outputs, labels)

        # backward + optimize
        loss.backward() # backward to get gradient values
        optimizer.step() # does the update

        # zero the parameter gradients
        optimizer.zero_grad()
        
        # accumulate loss
        running_loss += loss.item()
    if ((epoch+1) % 10 == 0):
        print(f"Epoch {epoch+1}: loss = {loss.item()}")

Epoch 10: loss = 0.0005521992570720613
Epoch 20: loss = 9.695346670923755e-05
Epoch 30: loss = 4.097184137208387e-05
Epoch 40: loss = 1.5642981452401727e-05
Epoch 50: loss = 5.987505119264824e-06
Epoch 60: loss = 5.879110176465474e-06
Epoch 70: loss = 3.3107405670307344e-06
Epoch 80: loss = 2.88267619907856e-06
Epoch 90: loss = 2.6551006158115342e-06
Epoch 100: loss = 1.1812525144705432e-06


## Validation 

In [183]:
correct = 0
total = 0
# since we're not training, we don't need to calculate the gradients for our outputs
with torch.no_grad():
    for data in validloader:
        inputs, labels = data
        inputs = torch.transpose(inputs, 1, 3)
        # calculate outputs by running images through the network
        outputs = cnn(inputs)
        # the class with the highest energy is what we choose as prediction
        _, predicted = torch.max(outputs.data, dim=1)
        _, label_indeces = torch.max(labels.data, dim=1)
        total += labels.size(0)
        correct += (predicted == label_indeces).sum().item()

print(f'Accuracy of the network on validation set: {100 * correct // total}%')

Accuracy of the network on validation set: 98%


## Test

In [184]:
correct = 0
total = 0
# since we're not training, we don't need to calculate the gradients for our outputs
with torch.no_grad():
    for data in testloader:
        inputs, labels = data
        inputs = torch.transpose(inputs, 1, 3)
        # calculate outputs by running images through the network
        outputs = cnn(inputs)
        # the class with the highest energy is what we choose as prediction
        _, predicted = torch.max(outputs.data, dim=1)
        _, label_indeces = torch.max(labels.data, dim=1)
        total += labels.size(0)
        correct += (predicted == label_indeces).sum().item()

print(f'Accuracy of the network on test set: {100 * correct // total}%')

Accuracy of the network on test set: 33%
